In [2]:
import pandas as pd
import numpy as np
from itertools import *
pd.set_option('display.max_rows', 15)
pd.set_option('display.max_columns', None)

In [3]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
# for autoreload modules
%load_ext autoreload
%autoreload 2

In [6]:
from statistic import Statistic
from utils import UtilsKy
from analyzer import HelperAnalyzer, AnalyzerPrediction

In [7]:
from naive_bayes import Bayes

In [8]:
# kyw3
path_data = '/mnt/files/workdata/work/merchants/merchant_33_kyw3_2020-06-05/04_experiments/ex_01_some_teach/'
db_teach = pd.read_csv(UtilsKy.DB_TEACH_KYW3, dtype=str, encoding='cp1251')
db_test = pd.read_csv(UtilsKy.DB_TEST_KYW3, dtype=str, encoding='cp1251')
white = pd.read_csv(UtilsKy.WHITE_KYW3 , dtype=str)

In [9]:
str(list(db_teach))

"['amount', 'amount_deviation', 'bank_currency', 'bin', 'city', 'count_months_to_end_card', 'day_of_week', 'gender2', 'hour', 'id', 'is_city_resolved', 'is_gender_undefined', 'latitude', 'longitude', 'order_id', 'phone_2_norm', 'status']"

In [10]:
path_resolving_file = '/mnt/files/workdata/work/merchants/merchant_33_kyw3_2020-06-05/02_pure_data/kyw3_for_resolve.csv'
df_resolving = pd.read_csv(path_resolving_file, dtype=str, encoding='cp1251')

In [11]:
str(list(df_resolving))

"['order_id', 'type', 'status', 'date', 'processed', 'amount', 'currency', 'card_holder', 'email', 'phone', 'address', 'city_from_order', 'country', 'zip', 'ip', 'project_id', 'reported', 'status_chargeback', 'date_cb_only', 'card_masked', 'card_expire', 'amount_in_currency', 'date_only', 'id']"

In [12]:
db_teach = db_teach.merge(df_resolving[['order_id', 'country']], left_on='order_id', right_on='order_id', how='left')
db_test = db_test.merge(df_resolving[['order_id', 'country']], left_on='order_id', right_on='order_id', how='left')

In [13]:
str(list(db_test))

"['amount', 'amount_deviation', 'bank_currency', 'bin', 'city', 'count_months_to_end_card', 'day_of_week', 'gender2', 'hour', 'id', 'is_city_resolved', 'is_gender_undefined', 'latitude', 'longitude', 'order_id', 'phone_2_norm', 'status', 'country']"

In [14]:
COL_FACTORS = ['amount_deviation', 'bank_currency', 'bin', 'city', 'count_months_to_end_card', 'day_of_week',
                'hour', 'is_city_resolved', 'is_gender_undefined', 'phone_2_norm', 'country', 'status']

In [15]:
main_factors = ['amount_deviation', 'day_of_week', 'hour', 'is_city_resolved', 'is_gender_undefined', 'status']
extra_factors = ['city', 'bank_currency', 'count_months_to_end_card', 'country','phone_2_norm', 'bin' ]

In [16]:
result_df_amount = None
analyzer_prediction =  AnalyzerPrediction(db_teach, db_test, white)

In [17]:
teach = db_teach[COL_FACTORS].copy()
test = db_test[COL_FACTORS].copy()

In [18]:
model = Bayes()#calculation_type='amount'
model.fit(teach)
model.create_test_weight(test)

factor_list = ['amount_deviation', 'bank_currency', 'bin', 'city', 'count_months_to_end_card', 'day_of_week', 'hour', 'is_city_resolved', 'is_gender_undefined', 'phone_2_norm', 'country']


In [19]:
result_df_amount = None
for i in list(range(len(extra_factors)+1)):
    for comb_factors in combinations(extra_factors, i):
        comb_factors = list(comb_factors)
        factor_list = main_factors + comb_factors
        
        test_pred = model.predict_proba(col_names=factor_list)
        db_test["probability"] = test_pred[:, 1]  
        description = str(comb_factors)
        result_df_amount = analyzer_prediction.get_table_prediction(description=description, result_df=result_df_amount, metric="amount")


In [20]:
n = result_df_amount.shape[0]
sub_rows = list(range(n))[::2]
stat_best = result_df_amount.copy().iloc[sub_rows,:]

col_names = [col for col in stat_best.columns if col.startswith('p_') ] 
stat_best.loc[:, col_names] = stat_best.loc[:, col_names].astype(float)
stat_best = stat_best.sort_values(by="rating", ascending=False)

In [21]:
stat_best.iloc[:,:11]

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,rating
8,['country'],3.97,9.21,12.29,15.55,17.21,18.14,21.12,27.70,48.55,134.48
0,[],3.87,7.35,12.19,15.09,16.67,17.68,20.97,26.91,47.37,129.45
26,"['bank_currency', 'country']",3.53,8.33,12.29,15.20,16.95,18.14,21.12,26.82,44.88,128.00
4,['bank_currency'],3.43,7.35,12.19,15.09,16.67,17.68,21.01,26.72,44.54,125.99
70,"['bank_currency', 'country', 'phone_2_norm']",3.65,8.87,10.21,12.12,16.94,17.51,19.59,27.55,44.19,123.53
38,"['country', 'phone_2_norm']",3.65,8.87,10.21,12.12,16.50,17.51,19.24,27.55,43.77,122.67
28,"['bank_currency', 'phone_2_norm']",3.54,8.77,10.10,12.02,15.96,17.37,18.70,27.45,43.67,121.51
...,...,...,...,...,...,...,...,...,...,...,...
50,"['city', 'bank_currency', 'bin']",0.39,2.42,4.53,5.59,7.53,8.00,10.74,19.55,38.09,78.10
22,"['city', 'bin']",0.39,2.42,4.53,5.59,7.53,8.00,10.74,19.55,38.09,78.10


In [22]:
stat_best.iloc[0,0]

"['country']"